In [1]:
import numpy as np
from typing import Union
from scipy.sparse import csr_matrix, dok_matrix, csc_matrix, coo_matrix
from tqdm.notebook import tqdm
from symmer import PauliwordOp

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html#scipy.sparse.dok_matrix

In [2]:
M = dok_matrix((100, 100))

In [3]:
nq = 3

empty = dok_matrix((4**nq, 4**nq), 
                   dtype=bool)


In [ ]:
P = PauliwordOp.random(10,6)
# int_list = P.symp_matrix @ (1 << np.arange(P.symp_matrix.shape[1])[::-1])

int_list = P.symp_matrix @ (1 << np.arange(P.symp_matrix.shape[1])[::-1])



In [ ]:
# PauliwordOp(P.symp_matrix[sort_inds], P.coeff_vec[sort_inds])

In [ ]:
n_qubits = 2
((np.arange(2 ** n_qubits).reshape([-1, 1]) &
              (1 << np.arange(n_qubits))[::-1])) > 0

In [ ]:
def get_ij_operator(i,j,n_qubits,binary_vec=None, return_operator=False):
    """
    """
    if n_qubits > 30:
        raise ValueError('Too many qubits, might run into memory limitations.')

    if binary_vec is None:
        binary_vec = (
            ((np.arange(2 ** n_qubits).reshape([-1, 1]) & 
            (1 << np.arange(n_qubits))[::-1])) > 0
        ).astype(bool)

    left  = np.array([int(i) for i in np.binary_repr(i, width=n_qubits)]).astype(bool)
    right = np.array([int(i) for i in np.binary_repr(j, width=n_qubits)]).astype(bool)

    AND = left & right # AND where -1 sign
    XZX_sign_flips = (-1) ** np.sum(AND & binary_vec, axis=1) # XZX = -X multiplications
        
    if i != j:
        XOR = left ^ right # XOR where +-i phase

        XZ_mult = left & binary_vec
        ZX_mult = binary_vec & right

        XZ_phase = (-1j) ** np.sum(XZ_mult & ~ZX_mult, axis=1) # XZ=-iY multiplications
        ZX_phase = (+1j) ** np.sum(ZX_mult & ~XZ_mult, axis=1) # ZX=+iY multiplications
        phase_mod = XZX_sign_flips * XZ_phase * ZX_phase
        
        ij_symp_matrix = np.hstack([np.tile(XOR, [2**n_qubits, 1]), binary_vec])
        coeffs = phase_mod/2**n_qubits
        
        if return_operator:
            ij_operator= PauliwordOp(ij_symp_matrix, phase_mod/2**n_qubits)
            return ij_operator
    else:
        ij_symp_matrix = np.hstack([np.zeros_like(binary_vec), binary_vec])
        coeffs = XZX_sign_flips/2**n_qubits 
        
        if return_operator:
            ij_operator= PauliwordOp(ij_symp_matrix, XZX_sign_flips/2**n_qubits)
            return ij_operator

    return ij_symp_matrix, coeffs #ij_operator

In [ ]:
#     def _from_matrix_projector(cls, 
#             matrix: Union[np.array, csr_matrix],
#             n_qubits: int
#         ) -> "PauliwordOp":
#         """
#         """
#         if isinstance(matrix, np.ndarray):
#             row, col = np.where(matrix)
#         elif isinstance(matrix, (csr_matrix, csc_matrix, coo_matrix)):
#             row, col = matrix.nonzero()
#         else:
#             raise ValueError('Unrecognised matrix type, must be one of np.array or sp.sparse.csr_matrix')
        
#         binary_vec = (
#             (
#                 np.arange(2 ** n_qubits).reshape([-1, 1]) & 
#                 (1 << np.arange(n_qubits))[::-1]
#             ) > 0
#         ).astype(bool)

#         P_out = cls.empty(n_qubits)
#         for i,j in tqdm(zip(row, col), desc='Building operator via projectors', total=len(row)):
#             ij_op = get_ij_operator(i,j,n_qubits,binary_vec=binary_vec) 
#             P_out += ij_op * matrix[i,j]

#         return P_out

In [ ]:
def from_matrix_projector( matrix: Union[np.array, csr_matrix], 
                           n_qubits: int) -> "PauliwordOp":
    """
    """
    assert n_qubits <=32, 'cannot decompose matrices above 32 qubits'
    
    if isinstance(matrix, np.ndarray):
        row, col = np.where(matrix)
    elif isinstance(matrix, (csr_matrix, csc_matrix, coo_matrix)):
        row, col = matrix.nonzero()
    else:
        raise ValueError('Unrecognised matrix type, must be one of np.array or sp.sparse.csr_matrix')
    
    sym_operator = dok_matrix((4**n_qubits, 2*n_qubits), 
                        dtype=bool)
    
    coeff_operator = dok_matrix((4**n_qubits, 1), 
                        dtype=complex)
    
    
    binary_vec = (
        (
            np.arange(2 ** n_qubits).reshape([-1, 1]) & 
            (1 << np.arange(n_qubits))[::-1]
        ) > 0).astype(bool)
    
    binary_convert = 1 << np.arange(2*n_qubits)[::-1]
    #P_out = cls.empty(n_qubits)
    for i,j in tqdm(zip(row, col), desc='Building operator via projectors', total=len(row)):
        
        ij_symp_matrix, proj_coeffs = get_ij_operator(i,j,
                                                      n_qubits,
                                                      binary_vec=binary_vec) 
        
        ### find location in symp matrix
        int_list = ij_symp_matrix @ binary_convert #(1 << np.arange(ij_symp_matrix.shape[1])[::-1])        
        
        # populate sparse mats
        sym_operator[int_list, :] = ij_symp_matrix
        coeff_operator[int_list] += proj_coeffs.reshape(-1,1) * matrix[i,j]
    
    
    ### only keep nonzero coeffs! (skips expensive cleanup)
    nonzero = coeff_operator.nonzero()[0]
    P_out = PauliwordOp(sym_operator[nonzero,:].toarray(), 
                        coeff_operator[nonzero].toarray()[:,0])
    
    # P_out = PauliwordOp(sym_operator.toarray(),
    #                    coeff_operator.toarray()[:,0]).cleanup()
    return P_out

In [ ]:
nq = 2
random_M = np.random.random((2**nq,2**nq))


# nq=2
# random_M = np.array([[ 2.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
#        [ 0.+0.j, -2.+0.j,  0.+0.j,  1.+0.j],
#        [ 1.+0.j,  0.+0.j, -2.+0.j,  0.+0.j],
#        [ 0.+0.j,  1.+0.j,  0.+0.j,  2.+0.j]])

In [ ]:
P_test = from_matrix_projector(random_M, nq)

P_test

In [ ]:
print(np.allclose(P_test.to_sparse_matrix.toarray(),
            random_M))

In [ ]:
P_old = PauliwordOp.from_matrix(random_M)

np.allclose(P_old.to_sparse_matrix.toarray(),
            random_M)

In [ ]:
P_old

In [ ]:
# test = PauliwordOp(symp.toarray(), coeff.toarray()[:,0]).cleanup()
# test

In [ ]:
PauliwordOp.from_dictionary({'XI':1,
                             'ZZ':2}).to_sparse_matrix.toarray()

In [15]:
PauliwordOp.from_dictionary({'XX':1,
                             'ZZ':2,
                              'YY':2,
                             'YI':-1}).to_sparse_matrix.toarray()

array([[ 2.+0.j,  0.+0.j,  0.+1.j, -1.+0.j],
       [ 0.+0.j, -2.+0.j,  3.+0.j,  0.+1.j],
       [ 0.-1.j,  3.+0.j, -2.+0.j,  0.+0.j],
       [-1.+0.j,  0.-1.j,  0.+0.j,  2.+0.j]])

In [25]:
mat = np.array([[1,0,0,0],
                [1,0,0,0],
                [1,0,-1j,0],
                [1,0,0,0]])
sparse = csr_matrix(mat)
PauliOp_from_matrix = PauliwordOp.from_matrix(sparse, strategy='full_basis')
PauliOp_from_matrix.to_dictionary

Building operator via full basis:   0%|          | 0/16 [00:00<?, ?it/s]

{'II': (0.25-0.25j),
 'IZ': (0.25-0.25j),
 'ZI': (0.25+0.25j),
 'ZZ': (0.25+0.25j),
 'IX': (0.25+0j),
 'IY': (-0-0.25j),
 'ZX': (0.25+0j),
 'ZY': (-0-0.25j),
 'XI': (0.25+0j),
 'XZ': (0.25+0j),
 'YI': (-0-0.25j),
 'YZ': (-0-0.25j),
 'XX': (0.25+0j),
 'XY': (-0-0.25j),
 'YX': (-0-0.25j),
 'YY': (-0.25+0j)}

In [4]:
from scipy.sparse import rand
import numpy as np
from time import time

n_qubits = 5#8
density = 0.1

dim = 2**n_qubits
#x = rand(D, D, density=1/(2**(1.9*n_qubits)), format='csr')
sparse_mat = rand(dim, dim, 
                  density=density, 
                  format='csr')

sparse_mat = sparse_mat.toarray()

# n_qubits=2
# sparse_mat = np.random.random((2**n_qubits,2**n_qubits))
# sparse_mat = sparse_mat.astype(complex)


# n_qubits=3
# sparse_mat = PauliwordOp.from_dictionary({'ZZI':2,
#                                           'YII':2j+1}).to_sparse_matrix

In [5]:
P_test = from_matrix_projector(sparse_mat, n_qubits)

NameError: name 'from_matrix_projector' is not defined

In [ ]:
np.allclose(sparse_mat,
           P_test.to_sparse_matrix.toarray())

In [6]:
Pold_proj = PauliwordOp.from_matrix(sparse_mat, strategy='projector')

Building operator via projectors:   0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:
P_test == Pold_proj

In [7]:
Pold_fullbasis = PauliwordOp.from_matrix(sparse_mat, 
                                         strategy='full_basis',
                                        operator_basis = None)

Building operator via full basis:   0%|          | 0/1024 [00:00<?, ?it/s]

In [8]:
Pold_fullbasis  == Pold_proj

True

In [ ]:
Y_sign = (Pold_fullbasis.Y_count%2 * -2)+1
Pold_fullbasis.coeff_vec = Pold_fullbasis.coeff_vec * Y_sign

Pold_fullbasis  == Pold_proj

In [ ]:
Pold_proj

In [ ]:
op_basis[-1].Y_count % 2

In [ ]:
abs(P_test.coeff_vec) - abs(Pold_fullbasis.coeff_vec)

In [ ]:
### GENERATE operator basis!
n_qubits=2

int_list = np.arange(4 ** (n_qubits))
XZ_block = (((int_list[:, None] & (1 << np.arange(2 * n_qubits))[::-1])) > 0).astype(bool)
op_basis = PauliwordOp(XZ_block, np.ones(XZ_block.shape[0]))
op_basis

In [ ]:
XX = dok_matrix((16,1),
                dtype=complex)

In [ ]:
# XX[[2,3],0] = 

In [ ]:
np.array([2,1]).shape

In [ ]:
XX[[2,3]] = np.array([1,2], dtype=complex)

In [9]:
op = PauliwordOp.from_dictionary({'Y':1})

nq = op.n_qubits
matrix = np.random.random((2**nq,2**nq))
denominator = 2 ** n_qubits

const = np.einsum(
    'ij,ij->', 
    op.to_sparse_matrix.toarray(), 
    matrix, 
    optimize=True
) / denominator

In [10]:
const

-0.020525211510709105j

-0.020525211510709105j

In [ ]:
.toarray() * matrix

In [ ]:
matrix = csr_matrix(matrix)

const = (op.to_sparse_matrix.multiply(matrix)).sum() / denominator

const